# Unrolling for-loops
https://github.com/Qiskit/qiskit-terra/pull/9670

In [2]:
from qiskit.circuit import *

In [3]:
qreg, creg = QuantumRegister(5, "q"), ClassicalRegister(2, "c")

body = QuantumCircuit(3, 1)
loop_parameter = Parameter("foo")
indexset = range(0, 10, 2)

body.rx(loop_parameter, [0, 1, 2])

circuit = QuantumCircuit(qreg, creg)
circuit.for_loop(indexset, loop_parameter, body, [1, 2, 3], [1])
print(circuit.draw())

                  
q_0: ─────────────
     ┌───────────┐
q_1: ┤0          ├
     │           │
q_2: ┤1          ├
     │           │
q_3: ┤2          ├
     │  For_loop │
q_4: ┤           ├
     │           │
c_0: ╡           ╞
     │           │
c_1: ╡0          ╞
     └───────────┘


In [4]:
from qiskit.qasm3 import dumps
print(dumps(circuit))

OPENQASM 3;
include "stdgates.inc";
bit[2] c;
qubit[5] _all_qubits;
let q = _all_qubits[0:4];
for foo in [0:2:9] {
  rx(foo) q[1];
  rx(foo) q[2];
  rx(foo) q[3];
}



In [5]:
from qiskit.transpiler.passes import UnrollForLoops
circuit_unroll = UnrollForLoops()(circuit)

In [6]:
print(dumps(circuit_unroll))
circuit_unroll.draw()

OPENQASM 3;
include "stdgates.inc";
bit[2] c;
qubit[5] _all_qubits;
let q = _all_qubits[0:4];
rx(0) q[1];
rx(2) q[1];
rx(4) q[1];
rx(6) q[1];
rx(8) q[1];
rx(0) q[2];
rx(2) q[2];
rx(4) q[2];
rx(6) q[2];
rx(8) q[2];
rx(0) q[3];
rx(2) q[3];
rx(4) q[3];
rx(6) q[3];
rx(8) q[3];



q_0: ─────────────────────────────────────────────
     ┌───────┐┌───────┐┌───────┐┌───────┐┌───────┐
q_1: ┤ Rx(0) ├┤ Rx(2) ├┤ Rx(4) ├┤ Rx(6) ├┤ Rx(8) ├
     ├───────┤├───────┤├───────┤├───────┤├───────┤
q_2: ┤ Rx(0) ├┤ Rx(2) ├┤ Rx(4) ├┤ Rx(6) ├┤ Rx(8) ├
     ├───────┤├───────┤├───────┤├───────┤├───────┤
q_3: ┤ Rx(0) ├┤ Rx(2) ├┤ Rx(4) ├┤ Rx(6) ├┤ Rx(8) ├
     └───────┘└───────┘└───────┘└───────┘└───────┘
q_4: ─────────────────────────────────────────────
                                                  
c: 2/═════════════════════════════════════════════

In [7]:
print(dumps(UnrollForLoops(max_target_depth=4)(circuit)))

OPENQASM 3;
include "stdgates.inc";
bit[2] c;
qubit[5] _all_qubits;
let q = _all_qubits[0:4];
for foo in [0:2:9] {
  rx(foo) q[1];
  rx(foo) q[2];
  rx(foo) q[3];
}



In [8]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pass_manager = generate_preset_pass_manager(1)
pass_manager.run(circuit).draw()

q_0: ─────────────
     ┌───────────┐
q_1: ┤0          ├
     │           │
q_2: ┤1          ├
     │           │
q_3: ┤2          ├
     │  For_loop │
q_4: ┤           ├
     │           │
c_0: ╡           ╞
     │           │
c_1: ╡0          ╞
     └───────────┘

In [9]:
pass_manager.pre_optimization.append(UnrollForLoops())
pass_manager.run(circuit).draw()

global phase: π
                            
q_0: ───────────────────────
     ┌─────────────────────┐
q_1: ┤ U3(1.1504,-π/2,π/2) ├
     ├─────────────────────┤
q_2: ┤ U3(1.1504,-π/2,π/2) ├
     ├─────────────────────┤
q_3: ┤ U3(1.1504,-π/2,π/2) ├
     └─────────────────────┘
q_4: ───────────────────────
                            
c: 2/═══════════════════════

In [10]:
# skip when continue and break
import math
from qiskit import QuantumCircuit
qc = QuantumCircuit(2, 1)

with qc.for_loop(range(5)) as i:
    qc.rx(i * math.pi/4, 0)
    qc.cx(0, 1)
    qc.measure(0, 0)
    qc.break_loop().c_if(0, True)
print(dumps(qc))

OPENQASM 3;
include "stdgates.inc";
bit[1] c;
qubit[2] _all_qubits;
let q = _all_qubits[0:1];
for _loop_i_0 in [0:4] {
  rx(pi/4*_loop_i_0) q[0];
  cx q[0], q[1];
  c[0] = measure q[0];
  if (c[0] == 1) {
    break;
  }
}



In [11]:
print(dumps(UnrollForLoops()(qc)))

OPENQASM 3;
include "stdgates.inc";
bit[1] c;
qubit[2] _all_qubits;
let q = _all_qubits[0:1];
for _loop_i_0 in [0:4] {
  rx(pi/4*_loop_i_0) q[0];
  cx q[0], q[1];
  c[0] = measure q[0];
  if (c[0] == 1) {
    break;
  }
}

